<a href="https://colab.research.google.com/github/irham235/Belajar-machine-learning/blob/main/binary_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
!pip install kaggle

In [5]:
from google.colab import files

# upload kaggle.json
uploaded = files.upload()

Saving kaggle (3).json to kaggle (3).json


In [6]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/

In [7]:
!chmod 600 ~/.kaggle/kaggle.json

In [9]:
!kaggle datasets download -d marklvl/sentiment-labelled-sentences-data-set

  0% 0.00/326k [00:00<?, ?B/s]
100% 326k/326k [00:00<00:00, 92.0MB/s]


In [17]:
import zipfile
dataset_zip = zipfile.ZipFile('sentiment-labelled-sentences-data-set.zip', 'r')
dataset_zip.extractall()
dataset_zip.close()

Setelah dataset diunduh, kita load dataset pada Colaboratory. Dan pastinya kita buat dataframe dari dataset kita agar lebih mudah diproses.

In [20]:
import pandas as pd
df = pd.read_csv('/content/sentiment labelled sentences/yelp_labelled.txt', names=['sentence', 'label'], sep='\t')

Untuk melihat 5 sampel terakhir dari dataset kita, panggil fungsi tail().

In [22]:
df.tail()

,sentence,label
995,I think food should have flavor and texture an...,0
996,Appetite instantly gone.,0
997,Overall I was not impressed and would not go b...,0
998,"The whole experience was underwhelming, and I ...",0
999,"Then, as if I hadn't wasted enough of my life ...",0


Setelah itu kita bagi dataset menjadi train dan test set.

In [35]:
from sklearn.model_selection import train_test_split
kalimat = df['sentence'].values
y = df['label'].values
kalimat_latih, kalimat_test, y_latih, y_test = train_test_split(kalimat, y, test_size=0.2)

Agar teks dapat dipahami oleh model, kita harus lakukan tokenisasi. Gunakan fungsi tokenizer pada data latih dan data test. Jangan lupa gunakan fungsi pad_sequences agar setiap sequence sama panjang.

In [37]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=250, oov_token='x')
tokenizer.fit_on_texts(kalimat_latih)
tokenizer.fit_on_texts(kalimat_test)

sekuens_latih = tokenizer.texts_to_sequences(kalimat_latih)
sekuens_test = tokenizer.texts_to_sequences(kalimat_test)

padded_latih = pad_sequences(sekuens_latih, maxlen=20)
padded_test = pad_sequences(sekuens_test, maxlen=20)

Untuk arsitektur yang akan digunakan adalah layer embedding, dengan argumen pertama sesuai dengan jumlah vocabulary/kata yang kita pakai pada tokenizer. Argumen selanjutnya adalah dimensi embedding, dan input_length yang merupakan panjang dari sequence. Nah di kita tidak menggunakan layer Flatten melainkan kita menggantinya dengan GlobalAveragePooling1D. Fungsi ini bekerja lebih baik pada kasus NLP dibanding Flatten.

In [40]:
import tensorflow as tf
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(250, 16, input_length=20),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid'),
])
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

Setelah arsitektur model dibentuk, dan loss function serta optimizer ditentukan, kita dapat memulai pelatihan model kita. Di sini saya menggunakan 30 epoch. Kamu bebas bereksperimen dengan nilai yang lain.

In [41]:
num_epochs = 30
history = model.fit(padded_latih, y_latih, epochs=num_epochs,
                    validation_data=(padded_test, y_test), verbose=2)

Epoch 1/30
25/25 - 1s - loss: 0.6925 - accuracy: 0.5213 - val_loss: 0.6925 - val_accuracy: 0.5400 - 1s/epoch - 42ms/step
Epoch 2/30
25/25 - 0s - loss: 0.6899 - accuracy: 0.5788 - val_loss: 0.6913 - val_accuracy: 0.5650 - 79ms/epoch - 3ms/step
Epoch 3/30
25/25 - 0s - loss: 0.6858 - accuracy: 0.6150 - val_loss: 0.6889 - val_accuracy: 0.5650 - 94ms/epoch - 4ms/step
Epoch 4/30
25/25 - 0s - loss: 0.6787 - accuracy: 0.6338 - val_loss: 0.6849 - val_accuracy: 0.5900 - 71ms/epoch - 3ms/step
Epoch 5/30
25/25 - 0s - loss: 0.6673 - accuracy: 0.6463 - val_loss: 0.6786 - val_accuracy: 0.6000 - 76ms/epoch - 3ms/step
Epoch 6/30
25/25 - 0s - loss: 0.6505 - accuracy: 0.6762 - val_loss: 0.6687 - val_accuracy: 0.6200 - 82ms/epoch - 3ms/step
Epoch 7/30
25/25 - 0s - loss: 0.6271 - accuracy: 0.7063 - val_loss: 0.6549 - val_accuracy: 0.6200 - 83ms/epoch - 3ms/step
Epoch 8/30
25/25 - 0s - loss: 0.5980 - accuracy: 0.7400 - val_loss: 0.6391 - val_accuracy: 0.6600 - 76ms/epoch - 3ms/step
Epoch 9/30
25/25 - 0s - l